In [1]:
from scipy.stats import kurtosis, iqr, skew, gmean, hmean, mode, normaltest, shapiro, ks_2samp
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from logging import getLogger, Formatter, StreamHandler, FileHandler, INFO, ERROR
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import os, gc, sys, time, random, math
from contextlib import contextmanager
from matplotlib import pyplot as plt
from IPython.display import display
from scipy import stats, special
from sklearn import set_config
from functools import partial
import lightgbm as lgb
import seaborn as sns
import pandas as pd
import typing as tp
import numpy as np
import warnings


warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
def init_logger():
    handler = StreamHandler()
    handler.setLevel(ERROR)
    handler.setFormatter(Formatter(LOGFORMAT))
    fh_handler = FileHandler('{}.log'.format(MODELNAME))
    fh_handler.setFormatter(Formatter(LOGFORMAT))
    logger.setLevel(ERROR)
    logger.addHandler(handler)
    logger.addHandler(fh_handler)
    
@contextmanager
def timer(name : tp.Text):
    t0 = time.time()
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')

COMPETITION = 'WIDS2021'
logger = getLogger(COMPETITION)
LOGFORMAT = '%(asctime)s %(levelname)s %(message)s'
MODELNAME = 'LGBMCV'
init_logger()

In [3]:
train = pd.read_csv('../input/widsdatathon2021/TrainingWiDS2021.csv', index_col=[0])
test = pd.read_csv('../input/widsdatathon2021/UnlabeledWiDS2021.csv', index_col=[0])
test_id = test.encounter_id.values
y = train.diabetes_mellitus.values
del train['diabetes_mellitus']

In [4]:
train = train.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})
test = test.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})
train.loc[train.age == 0, 'age'] = np.nan
train = train.drop(['readmission_status','encounter_id','hospital_id'], axis=1)
test = test.drop(['readmission_status','encounter_id','hospital_id'], axis=1)
train = train.replace([np.inf, -np.inf], np.nan)
test = test.replace([np.inf, -np.inf], np.nan)

In [5]:
min_max_feats=[f[:-4] for f in train.columns if f[-4:]=='_min']
for col in min_max_feats:
    train.loc[train[f'{col}_min'] > train[f'{col}_max'], [f'{col}_min', f'{col}_max']] = train.loc[train[f'{col}_min'] > train[f'{col}_max'], [f'{col}_max', f'{col}_min']].values
    test.loc[test[f'{col}_min'] > test[f'{col}_max'], [f'{col}_min', f'{col}_max']] = test.loc[test[f'{col}_min'] > test[f'{col}_max'], [f'{col}_max', f'{col}_min']].values


In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(train.head(2))
    display(train.describe())
    display(test.head(2))
    display(test.describe())


,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2fio2ratio_apache,arterial_ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
1,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
2,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,NaN,1,12.7,NaN,NaN,95.0,31.0,95.0,31.0,118.0,72.0,NaN,NaN,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,NaN,NaN,159.0,67.0,159.0,67.0,36.3,35.1,NaN,

,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2fio2ratio_apache,arterial_ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,125139.000000,125667.000000,130157.000000,128080.000000,130157.000000,130157.000000,126694.000000,51994.000000,128472.000000,129292.000000,130157.000000,130157.000000,47597.000000,104746.000000,105275.000000,30437.000000,127967.000000,127967.000000,129448.000000,127967.000000,115461.000000,129848.000000,103399.000000,130157.000000,129737.000000,30437.000000,30437.000000,30437.000000,30437.000000,129349.000000,105638.00000,123546.000000,66990.000000,130157.000000,100682.000000,35089.000000,35089.000000,129880.000000,129880.000000,128521.000000,128521.000000,129895.000000,129895.000000,35289.000000,35289.000000,129830.000000,129830.000000,127929.000000,127929.000000,129474.000000,129474.000000,129625.000000,129625.000000,35119.000000,35119.00000,129886.000000,129886.000000,128534.000000,128534.000000,125663.000000,125663.000000,25328.000000,25328.000000,124630.000000,124630.000000,118818.000000,118818.000000,126083.000000,126083.000000,25391.000000,25391.000000,123627.000000,123627.000000,116860.000000,116860.000000,123703.000000,123703.000000,123915.000000,123915.000000,25350.000000,25350.000000,124638.000000,124638.000000,118827.000000,118827.000000,100454.000000,100454.000000,58751.000000,58751.000000,5

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2fio2ratio_apache,arterial_ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
1,72,NaN,0,Caucasian,F,152.4,Floor,Accident & Emergency,82,admit,Med-Surg ICU,0.015278,NaN,2.8,110.0,104.01,0,0,1.9,44.0,1.49,NaN,4.0,6.0,0.0,5.0,97.0,38.0,39.9,0,54.0,NaN,NaN,NaN,NaN,31.0,130.0,36.4,NaN,0,5.4,NaN,NaN,104.0,40.0,104.0,40.0,66.0,38.0,NaN,NaN,123.0,54.0,123.0,54.0,35.0,15.0,100.0,96.0,NaN,NaN,149.0,76.0,149.0,76.0,38.7,35.6,NaN,NaN,59.0,59.0,59.0,59.0,46.0,44.0,NaN,NaN,80.0,80.0,80.0,80.0,19.0,19.0,97.0,96.0,NaN,NaN,116.0,116.0,116.0,116.0,36.4,36.4,2.8,2.8,1.9,1.9,44.0,40.0,9.8,9.3,1.49,1.44,104.0,97.0,26.0,23.0,14.5,14.2,42.8,39.9,NaN,NaN,NaN,NaN,173.0,173.0,5.7,4.9,132.0,130.0,5.6,5.4,2.8,2.8,1.9,1.9,40.0,40.0,9.8,9.8,1.49,1.49,104.0,104.0,26.0,26.0,14.5,14.5,42.8,42.8,NaN,NaN,NaN,NaN,173.0,173.0,4.9,4.9,132.0,132.0,5.6,5.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
2,86,NaN,0,Caucasian,F,175.3,Emergency Department,Accident & Emergency,82,admit,Med-Surg ICU,0.000000,NaN,NaN,117.0,106.01,0,0,NaN,19.0,0.92,NaN,4.0,6.0,0.0,5.0,73.0,116.0,NaN,0,41.0,NaN,NaN,NaN,NaN,53.0,142.0,36.3,NaN,0,NaN,NaN,NaN,101.0,27.0,101.0,27.0,116.0,56.0,NaN,NaN,129.0,41.0,129.0,41.0,51.0,14.0,100.0,95.0,NaN

,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2fio2ratio_apache,arterial_ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
count,10234.000000,9219.000000,10234.000000,9933.000000,10234.000000,10234.000000,9326.000000,4179.000000,10064.000000,10126.000000,10234.000000,10234.000000,3924.000000,8173.000000,8191.000000,2599.000000,10029.000000,10029.000000,10173.000000,10029.000000,9024.000000,10204.000000,8120.000000,10234.000000,10182.000000,2599.000000,2599.000000,2599.000000,2599.000000,10170.000000,8241.000000,9563.000000,5044.000000,10234.000000,7858.000000,2909.000000,2909.000000,10211.000000,10211.000000,10085.000000,10085.000000,10210.000000,10210.000000,2919.000000,2919.000000,10197.000000,10197.000000,10022.000000,10022.000000,10179.000000,10179.000000,10172.000000,10172.000000,2912.000000,2912.000000,10211.000000,10211.000000,10086.000000,10086.000000,9804.000000,9804.000000,2061.000000,2061.000000,9582.000000,9582.000000,9078.000000,9078.000000,9683.000000,9683.000000,2048.000000,2048.000000,9437.000000,9437.000000,8845.000000,8845.000000,9510.000000,9510.000000,9508.000000,9508.000000,2064.000000,2064.000000,9583.000000,9583.000000,9079.000000,9079.000000,7580.000000,7580.000000,4694.000000,4694.000000,4374.000000,4374.000000,9126.000000,9126.000000,8832.000000,8832.000000,9143.000000,9143.000000,9553.000000,9553.000000,

In [7]:
lbls = {}
for col in train.select_dtypes(exclude = np.number).columns.tolist():
    le = LabelEncoder().fit(pd.concat([train[col].astype(str),test[col].astype(str)]))   
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))
    lbls[col] = le
print('Categorical columns:', list(lbls.keys()))

Categorical columns: ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']


In [8]:
print(f'Percent of Nans in Train Data : {round(train.isna().sum().sum()/len(train), 2)}')
print(f'Percent of Nans in Test  Data : {round(test.isna().sum().sum()/len(test), 2)}')

Percent of Nans in Train Data : 61.33
Percent of Nans in Test  Data : 61.41


In [9]:
train['comorbidity_score'] = train['aids'].values * 23 + train['cirrhosis'] * 4  + train['hepatic_failure'] * 16 + train['immunosuppression'] * 10 + train['leukemia'] * 10 + train['lymphoma'] * 13 + train['solid_tumor_with_metastasis'] * 11
test['comorbidity_score'] = test['aids'].values * 23 + test['cirrhosis'] * 4  + test['hepatic_failure'] * 16 + test['immunosuppression'] * 10 + test['leukemia'] * 10 + test['lymphoma'] * 13 + test['solid_tumor_with_metastasis'] * 11
train['comorbidity_score'] = train['comorbidity_score'].fillna(0)
test['comorbidity_score'] = test['comorbidity_score'].fillna(0)
train['gcs_sum'] = train['gcs_eyes_apache']+train['gcs_motor_apache']+train['gcs_verbal_apache']
test['gcs_sum'] = test['gcs_eyes_apache']+test['gcs_motor_apache']+test['gcs_verbal_apache']
train['gcs_sum'] = train['gcs_sum'].fillna(0)
test['gcs_sum'] = test['gcs_sum'].fillna(0)
train['apache_2_diagnosis_type'] = train.apache_2_diagnosis.round(-1).fillna(-100).astype('int32')
test['apache_2_diagnosis_type'] = test.apache_2_diagnosis.round(-1).fillna(-100).astype('int32')
train['apache_3j_diagnosis_type'] = train.apache_3j_diagnosis.round(-2).fillna(-100).astype('int32')
test['apache_3j_diagnosis_type'] = test.apache_3j_diagnosis.round(-2).fillna(-100).astype('int32')
train['bmi_type'] = train.bmi.fillna(0).apply(lambda x: 5 * (round(int(x)/5)))
test['bmi_type'] = test.bmi.fillna(0).apply(lambda x: 5 * (round(int(x)/5)))
train['height_type'] = train.height.fillna(0).apply(lambda x: 5 * (round(int(x)/5)))
test['height_type'] = test.height.fillna(0).apply(lambda x: 5 * (round(int(x)/5)))
train['weight_type'] = train.weight.fillna(0).apply(lambda x: 5 * (round(int(x)/5)))
test['weight_type'] = test.weight.fillna(0).apply(lambda x: 5 * (round(int(x)/5)))
train['age_type'] = train.age.fillna(0).apply(lambda x: 10 * (round(int(x)/10)))
test['age_type'] = test.age.fillna(0).apply(lambda x: 10 * (round(int(x)/10)))
train['gcs_sum_type'] = train.gcs_sum.fillna(0).apply(lambda x: 2.5 * (round(int(x)/2.5))).divide(2.5)
test['gcs_sum_type'] = test.gcs_sum.fillna(0).apply(lambda x: 2.5 * (round(int(x)/2.5))).divide(2.5)
train['apache_3j_diagnosis_x'] = train['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
train['apache_2_diagnosis_x'] = train['apache_2_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
test['apache_3j_diagnosis_x'] = test['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
test['apache_2_diagnosis_x'] = test['apache_2_diagnosis'].astype('str').str.split('.',n=1,expand=True)[0]
train['apache_3j_diagnosis_split1'] = np.where(train['apache_3j_diagnosis'].isna() , np.nan , train['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[1]  )
test['apache_3j_diagnosis_split1']  = np.where(test['apache_3j_diagnosis'].isna() , np.nan , test['apache_3j_diagnosis'].astype('str').str.split('.',n=1,expand=True)[1]  )
train['apache_2_diagnosis_split1'] = np.where(train['apache_2_diagnosis'].isna() , np.nan , train['apache_2_diagnosis'].apply(lambda x : x % 10)  )
test['apache_2_diagnosis_split1']  = np.where(test['apache_2_diagnosis'].isna() , np.nan , test['apache_2_diagnosis'].apply(lambda x : x % 10) )

IDENTIFYING_COLS = ['age_type', 'height_type',  'ethnicity', 'gender', 'bmi_type'] 
train['profile'] = train[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis = 1)
test['profile'] = test[IDENTIFYING_COLS].apply(lambda x: hash(tuple(x)), axis = 1)
print(f'Number of unique Profiles : {train["profile"].nunique()}')


Number of unique Profiles : 6903


In [10]:
df = pd.concat([train['icu_id'], test['icu_id']])
agg = df.value_counts().to_dict()
train['icu_id_counts'] = np.log1p(train['icu_id'].map(agg))
test['icu_id_counts'] = np.log1p(test['icu_id'].map(agg))
df = pd.concat([train['age'], test['age']])
agg = df.value_counts().to_dict()
train['age_counts'] = np.log1p(train['age'].map(agg))
test['age_counts'] = np.log1p(test['age'].map(agg))
train["diff_bmi"] = train['bmi'].copy() 
train['bmi'] = train['weight']/((train['height']/100)**2)
train["diff_bmi"] = train["diff_bmi"]-train['bmi']
test["diff_bmi"] = test['bmi'].copy()
test['bmi'] = test['weight']/((test['height']/100)**2)
test["diff_bmi"] = test["diff_bmi"]-test['bmi']
train['pre_icu_los_days'] = train['pre_icu_los_days'].apply(lambda x:special.expit(x) )
test['pre_icu_los_days']  = test['pre_icu_los_days'].apply(lambda x:special.expit(x) )
train['abmi'] = train['age']/train['bmi']
train['agi'] = train['weight']/train['age']
test['abmi'] = test['age']/train['bmi']
test['agi'] = test['weight']/train['age']

In [11]:
d_cols = [c for c in train.columns if(c.startswith("d1"))]
h_cols = [c for c in train.columns if(c.startswith("h1"))]
train["dailyLabs_row_nan_count"] = train[d_cols].isna().sum(axis=1)
train["hourlyLabs_row_nan_count"] = train[h_cols].isna().sum(axis=1)
train["diff_labTestsRun_daily_hourly"] = train["dailyLabs_row_nan_count"] - train["hourlyLabs_row_nan_count"]
test["dailyLabs_row_nan_count"] = test[d_cols].isna().sum(axis=1)
test["hourlyLabs_row_nan_count"] = test[h_cols].isna().sum(axis=1)
test["diff_labTestsRun_daily_hourly"] = test["dailyLabs_row_nan_count"] - test["hourlyLabs_row_nan_count"]

In [12]:
lab_col = [c for c in train.columns if((c.startswith("h1")) | (c.startswith("d1")))]
lab_col_names = list(set(list(map(lambda i: i[ 3 : -4], lab_col))))

print("len lab_col",len(lab_col))
print("len lab_col_names",len(lab_col_names))
print("lab_col_names\n",lab_col_names)

len lab_col 128
len lab_col_names 32
lab_col_names
 ['sysbp', 'mbp_invasive', 'sysbp_noninvasive', 'pao2fio2ratio', 'bun', 'diasbp_invasive', 'hco3', 'wbc', 'hematocrit', 'potassium', 'resprate', 'hemaglobin', 'sodium', 'spo2', 'albumin', 'arterial_pco2', 'glucose', 'lactate', 'mbp_noninvasive', 'arterial_po2', 'inr', 'creatinine', 'sysbp_invasive', 'temp', 'diasbp_noninvasive', 'bilirubin', 'platelets', 'arterial_ph', 'heartrate', 'calcium', 'mbp', 'diasbp']


In [13]:
first_h = []
for v in lab_col_names:
    first_h.append(v+"_started_after_firstHour")
    colsx = [x for x in test.columns if v in x]
    train[v+"_nans"] = train.loc[:, colsx].isna().sum(axis=1)
    test[v+"_nans"] = test.loc[:, colsx].isna().sum(axis=1)
    train[v+"_d1_value_range"] = train[f"d1_{v}_max"].subtract(train[f"d1_{v}_min"])    
    train[v+"_h1_value_range"] = train[f"h1_{v}_max"].subtract(train[f"h1_{v}_min"])
    train[v+"_d1_h1_max_eq"] = (train[f"d1_{v}_max"]== train[f"h1_{v}_max"]).astype(np.int8)
    train[v+"_d1_h1_min_eq"] = (train[f"d1_{v}_min"]== train[f"h1_{v}_min"]).astype(np.int8)
    train[v+"_d1_zero_range"] = (train[v+"_d1_value_range"] == 0).astype(np.int8)
    train[v+"_h1_zero_range"] =(train[v+"_h1_value_range"] == 0).astype(np.int8)
    train[v+"_tot_change_value_range_normed"] = abs((train[v+"_d1_value_range"].div(train[v+"_h1_value_range"])))#.div(df[f"d1_{v}_max"]))
    train[v+"_started_after_firstHour"] = ((train[f"h1_{v}_max"].isna()) & (train[f"h1_{v}_min"].isna())) & (~train[f"d1_{v}_max"].isna())
    train[v+"_day_more_extreme"] = ((train[f"d1_{v}_max"]>train[f"h1_{v}_max"]) | (train[f"d1_{v}_min"]<train[f"h1_{v}_min"]))
    train[v+"_day_more_extreme"].fillna(False)    
    test[v+"_d1_value_range"] = test[f"d1_{v}_max"].subtract(test[f"d1_{v}_min"])   
    test[v+"_h1_value_range"] = test[f"h1_{v}_max"].subtract(test[f"h1_{v}_min"])
    test[v+"_d1_h1_max_eq"] = (test[f"d1_{v}_max"]== test[f"h1_{v}_max"]).astype(np.int8)
    test[v+"_d1_h1_min_eq"] = (test[f"d1_{v}_min"]== test[f"h1_{v}_min"]).astype(np.int8)
    test[v+"_d1_zero_range"] = (test[v+"_d1_value_range"] == 0).astype(np.int8)
    test[v+"_h1_zero_range"] =(test[v+"_h1_value_range"] == 0).astype(np.int8)
    test[v+"_tot_change_value_range_normed"] = abs((test[v+"_d1_value_range"].div(test[v+"_h1_value_range"])))
    test[v+"_started_after_firstHour"] = ((test[f"h1_{v}_max"].isna()) & (test[f"h1_{v}_min"].isna())) & (~test[f"d1_{v}_max"].isna())
    test[v+"_day_more_extreme"] = ((test[f"d1_{v}_max"]>test[f"h1_{v}_max"]) | (test[f"d1_{v}_min"]<test[f"h1_{v}_min"]))
    test[v+"_day_more_extreme"].fillna(False)

train["total_Tests_started_After_firstHour"] = train[first_h].sum(axis=1)
test["total_Tests_started_After_firstHour"] = test[first_h].sum(axis=1)
gc.collect()
train["total_Tests_started_After_firstHour"].describe()

count    130157.000000
mean          9.773996
std           5.746988
min           0.000000
25%           5.000000
50%          11.000000
75%          13.000000
max          31.000000
Name: total_Tests_started_After_firstHour, dtype: float64

In [14]:
groupers = ['apache_3j_diagnosis', 'profile']

for g in groupers:
    for v in lab_col_names:
        temp = pd.concat([train[[f"d1_{v}_max",g]], test[[f"d1_{v}_max",g]]], axis=0).groupby(g)[f"d1_{v}_max"].mean().to_dict()
        train[f'mean_diff_d1_{v}_{g}_max'] = train[f"d1_{v}_max"]-train[g].map(temp)
        test[f'mean_diff_d1_{v}_{g}_max'] = test[f"d1_{v}_max"]-test[g].map(temp)
        temp = pd.concat([train[[f"d1_{v}_min",g]], test[[f"d1_{v}_min",g]]], axis=0).groupby(g)[f"d1_{v}_min"].mean().to_dict()   
        train[f'mean_diff_d1_{v}_{g}_min'] = train[f"d1_{v}_min"]-train[g].map(temp)
        test[f'mean_diff_d1_{v}_{g}_min'] = test[f"d1_{v}_min"]-test[g].map(temp)
        temp = pd.concat([train[[f"h1_{v}_max",g]], test[[f"h1_{v}_max",g]]], axis=0).groupby(g)[f"h1_{v}_max"].mean().to_dict()   
        train[f'mean_diff_h1_{v}_{g}_max'] = train[f"h1_{v}_max"]-train[g].map(temp)
        test[f'mean_diff_h1_{v}_{g}_max'] = test[f"h1_{v}_max"]-test[g].map(temp)
        temp = pd.concat([train[[f"h1_{v}_min",g]], test[[f"h1_{v}_min",g]]], axis=0).groupby(g)[f"h1_{v}_min"].mean().to_dict()   
        train[f'mean_diff_h1_{v}_{g}_min'] = train[f"h1_{v}_min"]-train[g].map(temp)
        test[f'mean_diff_h1_{v}_{g}_min'] = test[f"h1_{v}_min"]-test[g].map(temp)
gc.collect()

0

In [15]:
train['diasbp_indicator'] = (
(train['d1_diasbp_invasive_max'] == train['d1_diasbp_max']) & (train['d1_diasbp_noninvasive_max']==train['d1_diasbp_invasive_max'])|
(train['d1_diasbp_invasive_min'] == train['d1_diasbp_min']) & (train['d1_diasbp_noninvasive_min']==train['d1_diasbp_invasive_min'])|
(train['h1_diasbp_invasive_max'] == train['h1_diasbp_max']) & (train['h1_diasbp_noninvasive_max']==train['h1_diasbp_invasive_max'])|
(train['h1_diasbp_invasive_min'] == train['h1_diasbp_min']) & (train['h1_diasbp_noninvasive_min']==train['h1_diasbp_invasive_min'])
).astype(np.int8)


train['mbp_indicator'] = (
(train['d1_mbp_invasive_max'] == train['d1_mbp_max']) & (train['d1_mbp_noninvasive_max']==train['d1_mbp_invasive_max'])|
(train['d1_mbp_invasive_min'] == train['d1_mbp_min']) & (train['d1_mbp_noninvasive_min']==train['d1_mbp_invasive_min'])|
(train['h1_mbp_invasive_max'] == train['h1_mbp_max']) & (train['h1_mbp_noninvasive_max']==train['h1_mbp_invasive_max'])|
(train['h1_mbp_invasive_min'] == train['h1_mbp_min']) & (train['h1_mbp_noninvasive_min']==train['h1_mbp_invasive_min'])
).astype(np.int8)

train['sysbp_indicator'] = (
(train['d1_sysbp_invasive_max'] == train['d1_sysbp_max']) & (train['d1_sysbp_noninvasive_max']==train['d1_sysbp_invasive_max'])|
(train['d1_sysbp_invasive_min'] == train['d1_sysbp_min']) & (train['d1_sysbp_noninvasive_min']==train['d1_sysbp_invasive_min'])|
 (train['h1_sysbp_invasive_max'] == train['h1_sysbp_max']) & (train['h1_sysbp_noninvasive_max']==train['h1_sysbp_invasive_max'])|
(train['h1_sysbp_invasive_min'] == train['h1_sysbp_min']) & (train['h1_sysbp_noninvasive_min']==train['h1_sysbp_invasive_min'])   
).astype(np.int8)

train['d1_mbp_invnoninv_max_diff'] = train['d1_mbp_invasive_max'] - train['d1_mbp_noninvasive_max']
train['h1_mbp_invnoninv_max_diff'] = train['h1_mbp_invasive_max'] - train['h1_mbp_noninvasive_max']
train['d1_mbp_invnoninv_min_diff'] = train['d1_mbp_invasive_min'] - train['d1_mbp_noninvasive_min']
train['h1_mbp_invnoninv_min_diff'] = train['h1_mbp_invasive_min'] - train['h1_mbp_noninvasive_min']
train['d1_diasbp_invnoninv_max_diff'] = train['d1_diasbp_invasive_max'] - train['d1_diasbp_noninvasive_max']
train['h1_diasbp_invnoninv_max_diff'] = train['h1_diasbp_invasive_max'] - train['h1_diasbp_noninvasive_max']
train['d1_diasbp_invnoninv_min_diff'] = train['d1_diasbp_invasive_min'] - train['d1_diasbp_noninvasive_min']
train['h1_diasbp_invnoninv_min_diff'] = train['h1_diasbp_invasive_min'] - train['h1_diasbp_noninvasive_min']
train['d1_sysbp_invnoninv_max_diff'] = train['d1_sysbp_invasive_max'] - train['d1_sysbp_noninvasive_max']
train['h1_sysbp_invnoninv_max_diff'] = train['h1_sysbp_invasive_max'] - train['h1_sysbp_noninvasive_max']
train['d1_sysbp_invnoninv_min_diff'] = train['d1_sysbp_invasive_min'] - train['d1_sysbp_noninvasive_min']
train['h1_sysbp_invnoninv_min_diff'] = train['h1_sysbp_invasive_min'] - train['h1_sysbp_noninvasive_min']

test['diasbp_indicator'] = (
(test['d1_diasbp_invasive_max'] == test['d1_diasbp_max']) & (test['d1_diasbp_noninvasive_max']==test['d1_diasbp_invasive_max'])|
(test['d1_diasbp_invasive_min'] == test['d1_diasbp_min']) & (test['d1_diasbp_noninvasive_min']==test['d1_diasbp_invasive_min'])|
(test['h1_diasbp_invasive_max'] == test['h1_diasbp_max']) & (test['h1_diasbp_noninvasive_max']==test['h1_diasbp_invasive_max'])|
(test['h1_diasbp_invasive_min'] == test['h1_diasbp_min']) & (test['h1_diasbp_noninvasive_min']==test['h1_diasbp_invasive_min'])
).astype(np.int8)


test['mbp_indicator'] = (
(test['d1_mbp_invasive_max'] == test['d1_mbp_max']) & (test['d1_mbp_noninvasive_max']==test['d1_mbp_invasive_max'])|
(test['d1_mbp_invasive_min'] == test['d1_mbp_min']) & (test['d1_mbp_noninvasive_min']==test['d1_mbp_invasive_min'])|
(test['h1_mbp_invasive_max'] == test['h1_mbp_max']) & (test['h1_mbp_noninvasive_max']==test['h1_mbp_invasive_max'])|
(test['h1_mbp_invasive_min'] == test['h1_mbp_min']) & (test['h1_mbp_noninvasive_min']==test['h1_mbp_invasive_min'])
).astype(np.int8)

test['sysbp_indicator'] = (
(test['d1_sysbp_invasive_max'] == test['d1_sysbp_max']) & (test['d1_sysbp_noninvasive_max']==test['d1_sysbp_invasive_max'])|
(test['d1_sysbp_invasive_min'] == test['d1_sysbp_min']) & (test['d1_sysbp_noninvasive_min']==test['d1_sysbp_invasive_min'])|
 (test['h1_sysbp_invasive_max'] == test['h1_sysbp_max']) & (test['h1_sysbp_noninvasive_max']==test['h1_sysbp_invasive_max'])|
(test['h1_sysbp_invasive_min'] == test['h1_sysbp_min']) & (test['h1_sysbp_noninvasive_min']==test['h1_sysbp_invasive_min'])   
).astype(np.int8)

test['d1_mbp_invnoninv_max_diff'] = test['d1_mbp_invasive_max'] - test['d1_mbp_noninvasive_max']
test['h1_mbp_invnoninv_max_diff'] = test['h1_mbp_invasive_max'] - test['h1_mbp_noninvasive_max']
test['d1_mbp_invnoninv_min_diff'] = test['d1_mbp_invasive_min'] - test['d1_mbp_noninvasive_min']
test['h1_mbp_invnoninv_min_diff'] = test['h1_mbp_invasive_min'] - test['h1_mbp_noninvasive_min']
test['d1_diasbp_invnoninv_max_diff'] = test['d1_diasbp_invasive_max'] - test['d1_diasbp_noninvasive_max']
test['h1_diasbp_invnoninv_max_diff'] = test['h1_diasbp_invasive_max'] - test['h1_diasbp_noninvasive_max']
test['d1_diasbp_invnoninv_min_diff'] = test['d1_diasbp_invasive_min'] - test['d1_diasbp_noninvasive_min']
test['h1_diasbp_invnoninv_min_diff'] = test['h1_diasbp_invasive_min'] - test['h1_diasbp_noninvasive_min']

test['d1_sysbp_invnoninv_max_diff'] = test['d1_sysbp_invasive_max'] - test['d1_sysbp_noninvasive_max']
test['h1_sysbp_invnoninv_max_diff'] = test['h1_sysbp_invasive_max'] - test['h1_sysbp_noninvasive_max']
test['d1_sysbp_invnoninv_min_diff'] = test['d1_sysbp_invasive_min'] - test['d1_sysbp_noninvasive_min']
test['h1_sysbp_invnoninv_min_diff'] = test['h1_sysbp_invasive_min'] - test['h1_sysbp_noninvasive_min']


for v in ['albumin','bilirubin','bun','glucose','hematocrit','pao2fio2ratio','arterial_ph','resprate','sodium','temp','wbc','creatinine']:
    train[f'{v}_indicator'] = (((train[f'{v}_apache']==train[f'd1_{v}_max']) & (train[f'd1_{v}_max']==train[f'h1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_max']) & (train[f'd1_{v}_max']==train[f'd1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_max']) & (train[f'd1_{v}_max']==train[f'h1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_max']) & (train[f'h1_{v}_max']==train[f'd1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_max']) & (train[f'h1_{v}_max']==train[f'h1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_max']) & (train[f'h1_{v}_max']==train[f'd1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_min']) & (train[f'd1_{v}_min']==train[f'd1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_min']) & (train[f'd1_{v}_min']==train[f'h1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'd1_{v}_min']) & (train[f'd1_{v}_min']==train[f'h1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_min']) & (train[f'h1_{v}_min']==train[f'h1_{v}_max'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_min']) & (train[f'h1_{v}_min']==train[f'd1_{v}_min'])) |
                 ((train[f'{v}_apache']==train[f'h1_{v}_min']) & (train[f'h1_{v}_min']==train[f'd1_{v}_max'])) 
                ).astype(np.int8)
    test[f'{v}_indicator'] = (((test[f'{v}_apache']==test[f'd1_{v}_max']) & (test[f'd1_{v}_max']==test[f'h1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_max']) & (test[f'd1_{v}_max']==test[f'd1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_max']) & (test[f'd1_{v}_max']==test[f'h1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_max']) & (test[f'h1_{v}_max']==test[f'd1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_max']) & (test[f'h1_{v}_max']==test[f'h1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_max']) & (test[f'h1_{v}_max']==test[f'd1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_min']) & (test[f'd1_{v}_min']==test[f'd1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_min']) & (test[f'd1_{v}_min']==test[f'h1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'd1_{v}_min']) & (test[f'd1_{v}_min']==test[f'h1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_min']) & (test[f'h1_{v}_min']==test[f'h1_{v}_max'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_min']) & (test[f'h1_{v}_min']==test[f'd1_{v}_min'])) |
                 ((test[f'{v}_apache']==test[f'h1_{v}_min']) & (test[f'h1_{v}_min']==test[f'd1_{v}_max'])) 
                ).astype(np.int8)

In [16]:
more_extreme_cols = [c for c in train.columns if(c.endswith("_day_more_extreme"))]
train["total_day_more_extreme"] = train[more_extreme_cols].sum(axis=1)
test["total_day_more_extreme"] = test[more_extreme_cols].sum(axis=1)
train["d1_resprate_div_mbp_min"] = train["d1_resprate_min"].div(train["d1_mbp_min"])
train["d1_resprate_div_sysbp_min"] = train["d1_resprate_min"].div(train["d1_sysbp_min"])
train["d1_lactate_min_div_diasbp_min"] = train["d1_lactate_min"].div(train["d1_diasbp_min"])
train["d1_heartrate_min_div_d1_sysbp_min"] = train["d1_heartrate_min"].div(train["d1_sysbp_min"])
train["d1_hco3_div"]= train["d1_hco3_max"].div(train["d1_hco3_min"])
train["d1_resprate_times_resprate"] = train["d1_resprate_min"].multiply(train["d1_resprate_max"])
train["left_average_spo2"] = (2*train["d1_spo2_max"] + train["d1_spo2_min"])/3
test["d1_resprate_div_mbp_min"] = test["d1_resprate_min"].div(test["d1_mbp_min"])
test["d1_resprate_div_sysbp_min"] = test["d1_resprate_min"].div(test["d1_sysbp_min"])
test["d1_lactate_min_div_diasbp_min"] = test["d1_lactate_min"].div(test["d1_diasbp_min"])
test["d1_heartrate_min_div_d1_sysbp_min"] = test["d1_heartrate_min"].div(test["d1_sysbp_min"])
test["d1_hco3_div"]= test["d1_hco3_max"].div(test["d1_hco3_min"])
test["d1_resprate_times_resprate"] = test["d1_resprate_min"].multiply(test["d1_resprate_max"])
test["left_average_spo2"] = (2*test["d1_spo2_max"] + test["d1_spo2_min"])/3
train["total_chronic"] = train[["aids","cirrhosis", 'hepatic_failure']].sum(axis=1)
train["total_cancer_immuno"] = train[[ 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].sum(axis=1)
test["total_chronic"] = test[["aids","cirrhosis", 'hepatic_failure']].sum(axis=1)
test["total_cancer_immuno"] = test[[ 'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].sum(axis=1)
train["has_complicator"] = train[["aids","cirrhosis", 'hepatic_failure',
                            'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].max(axis=1)
test["has_complicator"] = test[["aids","cirrhosis", 'hepatic_failure',
                            'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']].max(axis=1)
gc.collect()
train[["has_complicator","total_chronic","total_cancer_immuno","has_complicator"]].describe()


,has_complicator,total_chronic,total_cancer_immuno,has_complicator
count,130157.000000,130157.000000,130157.000000,130157.000000
mean,0.069040,0.030709,0.058015,0.069040
std,0.253523,0.214531,0.275306,0.253523
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,1.000000,3.000000,4.000000,1.000000


In [17]:
train['apache_3j'] = np.where(train['apache_3j_diagnosis_type']<0 , np.nan ,
                            np.where(train['apache_3j_diagnosis_type'] < 200, 'Cardiovascular' , 
                            np.where(train['apache_3j_diagnosis_type'] < 400, 'Respiratory' , 
                            np.where(train['apache_3j_diagnosis_type'] < 500, 'Neurological' , 
                            np.where(train['apache_3j_diagnosis_type'] < 600, 'Sepsis' , 
                            np.where(train['apache_3j_diagnosis_type'] < 800, 'Trauma' ,  
                            np.where(train['apache_3j_diagnosis_type'] < 900, 'Haematological' ,         
                            np.where(train['apache_3j_diagnosis_type'] < 1000, 'Renal/Genitourinary' ,         
                            np.where(train['apache_3j_diagnosis_type'] < 1200, 'Musculoskeletal/Skin disease' , 'Operative Sub-Diagnosis Codes' ))))))))
                                    )
test['apache_3j'] = np.where(test['apache_3j_diagnosis_type']<0 , np.nan ,
                            np.where(test['apache_3j_diagnosis_type'] < 200, 'Cardiovascular' , 
                            np.where(test['apache_3j_diagnosis_type'] < 400, 'Respiratory' , 
                            np.where(test['apache_3j_diagnosis_type'] < 500, 'Neurological' , 
                            np.where(test['apache_3j_diagnosis_type'] < 600, 'Sepsis' , 
                            np.where(test['apache_3j_diagnosis_type'] < 800, 'Trauma' ,  
                            np.where(test['apache_3j_diagnosis_type'] < 900, 'Haematological' ,         
                            np.where(test['apache_3j_diagnosis_type'] < 1000, 'Renal/Genitourinary' ,         
                            np.where(test['apache_3j_diagnosis_type'] < 1200, 'Musculoskeletal/Skin disease' , 'Operative Sub-Diagnosis Codes' ))))))))
                                    )

In [18]:
trainf = pd.read_pickle('../input/widsfeatures2021/X.pkl')
testf = pd.read_pickle('../input/widsfeatures2021/X_test.pkl')
trainf = trainf.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})
testf = testf.rename(columns={'pao2_apache':'pao2fio2ratio_apache','ph_apache':'arterial_ph_apache'})

In [19]:
train.shape, test.shape, trainf.shape, testf.shape

((130157, 814), (10234, 814), (130157, 865), (10234, 865))

In [20]:
col_order = train.columns.tolist()
train = train[col_order]
test = test[col_order]
col_order = trainf.columns.tolist()
trainf = trainf[col_order]
testf = testf[col_order]

In [21]:
train = pd.concat([trainf.reset_index(drop=True),train.reset_index(drop=True)], axis=1)
test =  pd.concat([testf.reset_index(drop=True),test.reset_index(drop=True)], axis=1)
train= train.fillna(0); test= test.fillna(0)
gc.collect()
train.shape, test.shape

((130157, 1679), (10234, 1679))

**Dropping Duplicated Columns**

In [22]:
Cols = list(train.columns)
for i,item in enumerate(train.columns):
    if item in train.columns[:i]: Cols[i] = "toDROP"
train.columns = Cols
test.columns = Cols
train = train.drop("toDROP",1)
test = test.drop("toDROP",1)
train.shape, test.shape

((130157, 1348), (10234, 1348))

Reordering Columns again!

In [23]:
col_order = train.columns.tolist()
train = train[col_order]
test = test[col_order]

In [24]:
drop_cols = ['abmi', 'age_type', 'aids', 'albumin_apache', 'albumin_h1_value_range', 'albumin_h1_zero_range','albumin_tot_change_value_range_normed', 'apache_3j_diagnosis-cat_age', 'apache_post_operative','apache_post_operative_std_d1_temp_max', 'arf_apache_std_d1_hemaglobin_max', 'arterial_pco2_d1_h1_max_eq','arterial_pco2_d1_h1_min_eq', 'arterial_pco2_d1_zero_range', 'arterial_pco2_h1_zero_range','arterial_ph_apache', 'arterial_ph_d1_h1_max_eq', 'arterial_ph_d1_value_range', 'arterial_ph_d1_zero_range','arterial_ph_h1_zero_range', 'arterial_po2_d1_h1_max_eq', 'arterial_po2_d1_h1_min_eq', 'arterial_po2_d1_value_range', 'bilirubin_h1_value_range', 'bilirubin_h1_zero_range','bilirubin_tot_change_value_range_normed', 'bmi_type', 'bun_d1_h1_max_eq', 'bun_d1_zero_range','bun_h1_value_range', 'bun_h1_zero_range', 'calcium_d1_zero_range', 'calcium_h1_value_range','calcium_h1_zero_range', 'creatinine_h1_zero_range', 'd1_albumin_min', 'd1_arterial_pco2_min','d1_arterial_ph_max', 'd1_arterial_ph_min', 'd1_calcium_max', 'd1_diasbp_max', 'd1_diasbp_min','d1_hematocrit_min', 'd1_inr_max', 'd1_inr_min', 'd1_mbp_invasive_max', 'd1_mbp_invasive_min', 'd1_mbp_max','d1_mbp_min', 'd1_mbp_noninvasive_max', 'd1_mbp_noninvasive_min', 'd1_pao2fio2ratio_max', 'd1_pao2fio2ratio_min', 'd1_platelets_max', 'd1_resprate_max', 'd1_sysbp_invasive_min', 'd1_temp_min','d1_wbc_min', 'diasbp_d1_h1_max_eq', 'diasbp_d1_zero_range', 'diasbp_invasive_d1_h1_max_eq','diasbp_invasive_d1_value_range', 'diasbp_invasive_d1_zero_range', 'diasbp_invasive_h1_value_range','diasbp_invasive_h1_zero_range', 'diasbp_noninvasive_d1_h1_max_eq', 'diasbp_noninvasive_d1_zero_range','diasbp_noninvasive_h1_zero_range', 'diff_bmi', 'elective_surgery_mean_d1_sysbp_min', 'gcs_unable_apache','h1_albumin_max', 'h1_albumin_min', 'h1_arterial_pco2_max', 'h1_arterial_pco2_min', 'h1_arterial_ph_min','h1_arterial_po2_max', 'h1_bilirubin_max', 'h1_bun_max', 'h1_creatinine_min', 'h1_diasbp_noninvasive_max','h1_heartrate_max', 'h1_heartrate_min', 'h1_hemaglobin_min', 'h1_hematocrit_max', 'h1_hematocrit_min','h1_lactate_max', 'h1_lactate_min', 'h1_mbp_invasive_max', 'h1_mbp_invasive_min', 'h1_mbp_max', 'h1_mbp_min','h1_mbp_noninvasive_max', 'h1_mbp_noninvasive_min', 'h1_pao2fio2ratio_max', 'h1_pao2fio2ratio_min','h1_platelets_max', 'h1_platelets_min', 'h1_resprate_max', 'h1_resprate_min', 'h1_sodium_max','h1_spo2_max', 'h1_spo2_min', 'h1_sysbp_max', 'h1_sysbp_min', 'h1_sysbp_noninvasive_max','h1_sysbp_noninvasive_min', 'h1_temp_max', 'h1_temp_min', 'h1_wbc_max', 'h1_wbc_min', 'hco3_d1_h1_max_eq','hco3_d1_h1_min_eq', 'hco3_h1_value_range', 'hco3_h1_zero_range', 'heartrate_d1_zero_range','heartrate_h1_zero_range', 'height', 'hemaglobin_d1_value_range', 'hemaglobin_d1_zero_range','hematocrit_apache', 'hematocrit_d1_h1_min_eq', 'hematocrit_d1_value_range', 'hematocrit_d1_zero_range','inr_d1_h1_max_eq', 'inr_d1_h1_min_eq', 'inr_d1_value_range', 'inr_d1_zero_range', 'inr_day_more_extreme','inr_h1_value_range', 'inr_h1_zero_range', 'inr_started_after_firstHour', 'intubated_apache_mean_d1_spo2_max','lactate_h1_value_range', 'lactate_h1_zero_range', 'lymphoma', 'map_apache', 'mbp_d1_zero_range','mbp_h1_zero_range', 'mbp_invasive_d1_h1_min_eq', 'mbp_invasive_d1_value_range', 'mbp_invasive_d1_zero_range','mbp_invasive_h1_zero_range', 'mbp_noninvasive_d1_h1_max_eq', 'mbp_noninvasive_d1_h1_min_eq','mbp_noninvasive_d1_zero_range', 'mbp_noninvasive_h1_zero_range', 'mean_diff_d1_inr_min','mean_diff_h1_bilirubin_min', 'mean_diff_h1_inr_max', 'paco2_apache', 'paco2_for_ph_apache','pao2fio2ratio_apache', 'pao2fio2ratio_h1_value_range', 'pao2fio2ratio_h1_zero_range','rank_frqenc_leukemia', 'wbc_h1_value_range','platelets_d1_value_range', 'platelets_h1_zero_range', 'potassium_d1_h1_max_eq','potassium_h1_value_range','potassium_h1_zero_range', 'rank_frqenc_apache_2_diagnosis', 'resprate_apache', 'resprate_d1_h1_min_eq','resprate_d1_zero_range', 'sodium_d1_h1_min_eq', 'sodium_d1_zero_range','spo2_d1_h1_max_eq','sysbp_d1_zero_range', 'sysbp_h1_zero_range', 'sysbp_invasive_d1_h1_min_eq','sysbp_invasive_d1_zero_range','sysbp_noninvasive_d1_h1_min_eq', 'sysbp_noninvasive_d1_zero_range','sysbp_noninvasive_h1_zero_range','temp_d1_zero_range', 'ventilated_apache_std_d1_glucose_min', 'wbc_apache', 'wbc_d1_h1_min_eq','wbc_d1_value_range', 'wbc_d1_zero_range', 'wbc_h1_zero_range', 'gcs_eyes_apache_mean_d1_bun_min','rank_frqenc_aids']

In [25]:
drop_cols = list(set(drop_cols))
print(len(drop_cols))

185


In [26]:
cats = ['elective_surgery', 'icu_id', 'arf_apache', 'intubated_apache', 'ventilated_apache', 'cirrhosis','hepatic_failure', 'immunosuppression', 'leukemia', 'solid_tumor_with_metastasis', 'apache_3j_diagnosis_x','apache_2_diagnosis_x', 'apache_3j', 'apache_3j_diagnosis_split1', 'apache_2_diagnosis_split1', 'gcs_sum_type','hospital_admit_source', 'glucose_rate', 'glucose_wb', 'gcs_eyes_apache', 'glucose_normal', 'total_cancer_immuno','gender', 'total_chronic', 'icu_stay_type', 'apache_2_diagnosis_type', 'apache_3j_diagnosis_type']
len(cats)

27

In [27]:
for col in cats:
    train_only = list(set(train[col].unique()) - set(test[col].unique()))
    test_only = list(set(test[col].unique()) - set(train[col].unique()))
    both = list(set(test[col].unique()).union(set(train[col].unique())))
    train.loc[train[col].isin(train_only), col] = np.nan
    test.loc[test[col].isin(test_only), col] = np.nan
    try:
        lbl = OrdinalEncoder(dtype='int')
        train[col] = lbl.fit_transform(train[col].astype('str').values.reshape(-1,1))
        test[col] = lbl.transform(test[col].astype('str').values.reshape(-1,1))
    except:
        lbl = OrdinalEncoder(dtype='int')
        train[col] = lbl.fit_transform(train[col].astype('str').fillna('-1').values.reshape(-1,1))
        test[col] = lbl.transform(test[col].astype('str').fillna('-1').values.reshape(-1,1))
    temp = pd.concat([train[[col]], test[[col]]], axis=0)
    temp_mapping = temp.groupby(col).size()/len(temp)
    temp['enc'] = temp[col].map(temp_mapping)
    temp['enc'] = stats.rankdata(temp['enc'])
    temp = temp.reset_index(drop=True)
    train[f'rank_frqenc_{col}'] = temp[['enc']].values[:train.shape[0]]
    test[f'rank_frqenc_{col}'] = temp[['enc']].values[train.shape[0]:]               
    test[col] = test[col].astype('category')
    train[col] = train[col].astype('category')


In [28]:
drop_cols = list(set(drop_cols))
print(len(drop_cols))
train = train.drop(drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
train.shape, test.shape

185


((130157, 1183), (10234, 1183))

In [29]:
@contextmanager
def timer(logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None):
    if prefix: format_str = str(prefix) + format_str
    if suffix: format_str = format_str + str(suffix)
    start = time.time()
    yield
    d = time.time() - start
    out_str = format_str.format(d)
    if logger:
        logger.info(out_str)
    else:
        print(out_str)

In [30]:
class TreeModel:
    """Wrapper for LightGBM/XGBoost/CATBoost"""
    def __init__(self, model_type: str):
        self.model_type = model_type
        self.trn_data = None
        self.val_data = None
        self.model = None

    def train(self,
              params: dict,
              X_train: pd.DataFrame, y_train: np.ndarray,
              X_val: pd.DataFrame, y_val: np.ndarray,
              train_weight: tp.Optional[np.ndarray] = None,
              val_weight: tp.Optional[np.ndarray] = None,
              train_params: dict = {}):
        if self.model_type == "lgb":
            self.trn_data = lgb.Dataset(X_train, label=y_train, weight=train_weight)
            self.val_data = lgb.Dataset(X_val, label=y_val, weight=val_weight)
            self.model = lgb.train(params=params,
                                   train_set=self.trn_data,
                                   valid_sets=[self.trn_data, self.val_data],
                                   **train_params)
        elif self.model_type == "xgb":
            self.trn_data = xgb.DMatrix(X_train, y_train, weight=train_weight)
            self.val_data = xgb.DMatrix(X_val, y_val, weight=val_weight)
            self.model = xgb.train(params=params,
                                   dtrain=self.trn_data,
                                   evals=[(self.trn_data, "train"), (self.val_data, "val")],
                                   **train_params)
        elif self.model_type == "cat":
            self.trn_data = Pool(X_train, label=y_train, group_id=[0] * len(X_train))
            self.val_data =  Pool(X_val, label=y_val, group_id=[0] * len(X_val))
            self.model = CatBoost(params)
            self.model.fit(
                self.trn_data, eval_set=[self.val_data], use_best_model=True, **train_params)
        else:
            raise NotImplementedError

    def predict(self, X: pd.DataFrame):
        if self.model_type == "lgb":
            return self.model.predict(
                X, num_iteration=self.model.best_iteration)  # type: ignore
        elif self.model_type == "xgb":
            X_DM = xgb.DMatrix(X)
            return self.model.predict(
                X_DM, ntree_limit=self.model.best_ntree_limit)  # type: ignore
        elif self.model_type == "cat":
            return self.model.predict(X)
        else:
            raise NotImplementedError

    @property
    def feature_names_(self):
        if self.model_type == "lgb":
            return self.model.feature_name()
        elif self.model_type == "xgb":
            return list(self.model.get_score(importance_type="gain").keys())
        elif self.model_type == "cat":
             return self.model.feature_names_
        else:
            raise NotImplementedError

    @property
    def feature_importances_(self):
        if self.model_type == "lgb":
            return self.model.feature_importance(importance_type="gain")
        elif self.model_type == "xgb":
            return list(self.model.get_score(importance_type="gain").values())
        elif self.model_type == "cat":
            return self.model.feature_importances_
        else:
            raise NotImplementedError

In [31]:
ID_COL = "encounter_id"
#TGT_COL = "diabetes_mellitus"

N_SPLITS =10# 5
RANDOM_SEED = 190295 #,42,90295#42
USE_MODEL = "lgb"

MODEL_PARAMS = {
   "seed": RANDOM_SEED,
      "n_jobs": 4,
       'num_leaves': 250,
    'max_bin':550,
     'min_child_weight': 0.03454472573214212,
    'feature_fraction': 0.35,
    'bagging_fraction': 0.475,
    'min_data_in_leaf': 150,
     'max_depth': -1,
    'objective': 'binary',
    'learning_rate':  0.009867383057779643,
    "boosting_type": "gbdt",
    "metric": 'auc', 
     "bagging_seed": 7,
#"verbosity": -1,
'reg_alpha': 0.3899927210061127,
'reg_lambda': 0.6485237330340494,
##'subsample': 0.8032697250789377, 
##'colsample_bytree': 0.21067140508531404, 
##'learning_rate': 0.009867383057779643,
## 'reg_lambda': 10.987474846877767, 
##            'reg_alpha': 17.335285595031994,     
      ##
'cat_smooth': 39,
 ## parameters to keep the exactly the same
'subsample_for_bin': 200000,
'min_child_samples':285, #20,    
    
    
    
    }
TRAIN_PARAMS = {
    "num_boost_round": 30000,
    "early_stopping_rounds": 1000,
    "verbose_eval": 50,
}

In [32]:
#train[FEAT_COLS].to_pickle('train.pkl')
#test[FEAT_COLS].to_pickle('test.pkl')
#np.save('test_id.npy',test_id)
#np.save('y.npy',y)

In [33]:
#train[FEAT_COLS]

In [34]:
X = train
X_test = test 


kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
trn_val_indexs = list(kf.split(X, y))

In [35]:
oof_pred_arr = np.zeros(len(X))
test_preds_arr = np.zeros((N_SPLITS, len(X_test)))
feature_importances = pd.DataFrame()
score_list = []

In [36]:
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print("*" * 100)
    print(f"Fold: {fold}")

    X_trn = X.loc[trn_idx].reset_index(drop=True)
    X_val = X.loc[val_idx].reset_index(drop=True)
    y_trn = y[trn_idx]
    y_val = y[val_idx]

    model = TreeModel(model_type=USE_MODEL)
    with timer(prefix="Model training"):
        model.train(
            params=MODEL_PARAMS, X_train=X_trn, y_train=y_trn,
            X_val=X_val, y_val=y_val, train_params=TRAIN_PARAMS)
    fi_tmp = pd.DataFrame()
    fi_tmp["feature"] = model.feature_names_
    fi_tmp["importance"] = model.feature_importances_
    fi_tmp["fold"] = fold
    feature_importances = feature_importances.append(fi_tmp)

    val_pred = model.predict(X_val)
    score = roc_auc_score(y_val, val_pred)#, squared=False)
    #print(score)
    print(f"score: {score:.5f}")
    score_list.append([fold, score])
    oof_pred_arr[val_idx] = val_pred
    test_pred = model.predict(X_test)
    test_preds_arr[fold] = test_pred

****************************************************************************************************
Fold: 0
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=285 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=285 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Info] Number of positive: 25377, number of negative: 91764
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.848567 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 286948
[LightGBM] [Info] Number of data points in the train set: 117141, number of used features: 1183
[LightGBM] [Warning] min_data_in_leaf is set=150, min_child_samples=285 will be ignored. Current value: min_data_in_leaf=150
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.216636 -> initscore=-1.285377
[LightGBM] [Info] Start training from score -1.285377
Training until

In [37]:
oof_score = roc_auc_score(y, oof_pred_arr)#, squared=False)
score_list.append(["oof", oof_score])
pd.DataFrame(
    score_list, columns=["fold", "auc score"])

,fold,auc score
0,0,0.880285
1,1,0.877702
2,2,0.880243
3,3,0.885175
4,4,0.877724
5,5,0.882140
6,6,0.885518
7,7,0.878442
8,8,0.877560
9,9,0.880748


In [38]:
test_preds_arr = test_preds_arr.mean(axis=0)

In [39]:
#Create a  DataFrame
submission = pd.DataFrame({'encounter_id':test_id,'diabetes_mellitus':test_preds_arr})
                        

#Visualize the first 10 rows
submission.head(10)

,encounter_id,diabetes_mellitus
0,144740,0.046958
1,141990,0.252914
2,142038,0.086284
3,138628,0.050842
4,141682,0.275378
5,139096,0.024531
6,142994,0.785030
7,141954,0.036077
8,135344,0.603518
9,142552,0.866312


In [40]:
#Convert DataFrame to a csv file that can be uploaded
#This is saved in the same directory as your notebook
filename = 'submission.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: submission.csv


In [41]:
np.save('oof_preds.py',oof_pred_arr)
np.save('test_preds.py',test_preds_arr)